In [96]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import os

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import scipy.stats as stats

import seaborn as sns

from datetime import datetime

import re

from tqdm.notebook import tqdm

from tools import * 

In [97]:
import sys
import os

month_names = ['Gener','Febrer','Marc','Abril','Maig','Juny','Juliol','Agost','Setembre','Octubre','Novembre','Desembre']
months = range(1,13)
i2m = list(zip(months, month_names))


In [98]:
import dask.dataframe as dd
from dask.distributed import Client

In [99]:
client = Client()
client

/home/user/miniconda3/envs/tensorflow/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46345 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://127.0.0.1:37963 Dashboard: http://127.0.0.1:46345/status,Cluster Workers: 4 Cores: 16 Memory: 16.44 GB


In [100]:
def print_partitions(ddf:dd.core.DataFrame) -> None:
    for i in range(ddf.npartitions):
        print('Partion:', i)
        print(ddf.partitions[i].compute())

In [110]:
def read_dask_dataframe(folder_path:str, folder_type:str, config:dict, add_meta:bool=False) -> dd.core.DataFrame:
    assert folder_path != '' 
    assert folder_type != '' 
    assert not config.empty
    
    ddf = None
    
    if folder_type == 'csv':
        # re read file
        ddf = dd.read_csv(
            urlpath=f'{folder_path}/{config.year}/{config.dataset}/{config.year}_{config.month:02d}_{config.monthname}_{config.dataset}.{folder_type}',
            blocksize='default',
            lineterminator=None,
            compression='infer',
            sample=256000,
            enforce=False,
            assume_missing=False,
            storage_options=None,
            include_path_column=False,
            header=0
        )
    else: 
        raise 'Not supported yet'
    
    if add_meta: 
        ddf._name = f'{config.year}-{config.month}'
        # we have one partion
        # TODO
        # ddf.divisions = (0, ddf.shape[0].compute()-1)
    
    return ddf

In [111]:
config = pd.Series({
    'devide_by':60,
    'year':2021,
    'datafrom': 'BicingNou_ESTACIONS',
    'dataset': f'BicingNou_ESTACIONS_CLEAN',
    'ttl': 30,
    'month': 1,
    'monthname': 'Gener'
})

In [117]:
data_dd = read_dask_dataframe('../dades', 'csv', config) 

In [113]:
type(data_dd)

dask.dataframe.core.DataFrame

In [116]:
data_dd.compute().station_id.describe()

count    376086.000000
mean        261.563661
std         149.121159
min           1.000000
25%         133.000000
50%         263.000000
75%         389.000000
max         519.000000
Name: station_id, dtype: float64